In [1]:
# Cell 1: Setup & Dependencies
# =================================================================
# 1. Clone/Pull Repository (Ensures code is present in Colab runtime)
# =================================================================
import os

# Check if the folder already exists to avoid "already exists" errors
if not os.path.exists("adhd-assistant-capstone"):
    !git clone https://github.com/viveksahukar/adhd-assistant-capstone.git
else:
    print("Repository already cloned. Pulling latest changes...")
    %cd adhd-assistant-capstone
    !git pull

# 2. Navigate into the directory
%cd adhd-assistant-capstone

# 3. Install Dependencies
# Quiet output (-q) to keep the notebook clean
%pip install -q -r requirements.txt

print("✅ Environment Setup Complete.")

Cloning into 'adhd-assistant-capstone'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 61 (delta 31), reused 37 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (61/61), 29.08 KiB | 14.54 MiB/s, done.
Resolving deltas: 100% (31/31), done.
/content/adhd-assistant-capstone
✅ Environment Setup Complete.


In [2]:
# Cell 2: Project Initialization
# =================================================================
from setup_config import initialize_environment

# Your specific Project ID
PROJECT_ID = "adhd-assistant-capstone"

# Initialize Vertex AI and auth
# This returns the modules if you need to inspect them, but mainly sets up the global state.
auth, os, vertexai = initialize_environment(PROJECT_ID)

--- 🚀 Starting Cloud-Native Authentication ---
✅ Success! Authenticated using Colab Secrets.
Service Account: colab-runner@adhd-assistant-capstone.iam.gserviceaccount.com


In [3]:
# Cell 3: Import Agent Modules
# =================================================================
try:
    from agents import ConversationManagerAgent, TaskLogicAgent, ToolExecutionAgent
    print("✅ Successfully imported Agent classes.")
except ImportError as e:
    print(f"❌ Error importing agents: {e}")
    print("Make sure agents.py and tools.py are in the current folder.")

✅ Successfully imported Agent classes.


In [4]:
# Cell 4: Create Agent Instances
# =================================================================
# 1. Create the Specialist Agents
# Note: TaskLogicAgent now defaults to 'gemini-2.5-pro' per our refinement
task_agent = TaskLogicAgent()
tool_agent = ToolExecutionAgent()

# 2. Create the Supervisor (Conversation Manager)
# This injects the specialists into the coordinator
manager = ConversationManagerAgent(task_agent=task_agent, tool_agent=tool_agent)

print("✅ Agent System Online. Ready for user input.")

✅ Agent System Online. Ready for user input.


/usr/local/lib/python3.12/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [5]:
# Cell 5: Simulate User Interaction
# =================================================================

# 1. Define the User's "Brain Dump"
user_input = (
    "I have a big presentation on Friday morning at 10 AM. "
    "I need to call the doctor sometime next week to make an appointment, "
    "and I also have to remember to buy eggs and bread today."
)

print(f"👤 USER SAYS:\n{user_input}\n")
print("-" * 60)

# 2. Handle the message (with auto_confirm=False to test HITL)
# This mimics the "Think" step where the agent plans but waits for approval
agent_turn = manager.handle_user_message(
    user_text=user_input,
    user_id="notebook_test_user_01",
    auto_confirm=False
)

# 3. Display the Agent's proposed response
print(f"🤖 AGENT RESPONDS:\n{agent_turn.user_facing_message}")

👤 USER SAYS:
I have a big presentation on Friday morning at 10 AM. I need to call the doctor sometime next week to make an appointment, and I also have to remember to buy eggs and bread today.

------------------------------------------------------------
--- TOOL: Fetching context for user: 'notebook_test_user_01' ---
Error calling model or parsing response: 403 Vertex AI API has not been used in project adhd-assistant-capstone before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/aiplatform.googleapis.com/overview?project=adhd-assistant-capstone then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry. [reason: "SERVICE_DISABLED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "serviceTitle"
  value: "Vertex AI API"
}
metadata {
  key: "containerInfo"
  value: "adhd-assistant-capstone"
}
metadata {
  key: "consumer"
  value

In [6]:
# Cell 6: User Confirmation & Tool Execution
# =================================================================

if agent_turn.requires_confirmation:
    print("✅ USER ACTION: Confirming plan...\n")

    # Execute the pending actions using the Tool Execution Agent
    # This mimics the "Act" step
    results = tool_agent.execute_actions(agent_turn.pending_actions)

    print("--- 🛠️ Tool Execution Results ---")
    for res in results:
        # Check for success/failure in the tool output
        status = res.get("status", "unknown")
        details = res.get("details", str(res))
        print(f"[{status.upper()}] {details}")

else:
    print("ℹ️ No actions required confirmation.")

✅ USER ACTION: Confirming plan...

--- TOOL: Setting reminder: 'I have a big presentation on Friday morning at 10 AM. I need to call the doctor sometime next week to make an appointment, and I also have to remember to buy eggs and bread today.' at 1 hour from now ---
--- 🛠️ Tool Execution Results ---
[SUCCESS] Reminder for 'I have a big presentation on Friday morning at 10 AM. I need to call the doctor sometime next week to make an appointment, and I also have to remember to buy eggs and bread today.' set for 1 hour from now.
